# WhatsAppPush StatusLogs Notebook
Arunasish Ghosh

In [ ]:
import requests, json, uuid, time, csv, io

BASE_URL = 'https://e0cfn0j28h.execute-api.us-east-1.amazonaws.com/dev'
GET_SIGNED_URL = BASE_URL + '/upload-url/{user_id}'
CREATE_NOTIFICATION_URL = BASE_URL + '/notification'
LIST_STATUS_LOGS_URL = BASE_URL + '/status/{notification_id}'
DETAILS_STATUS_LOG_URL = BASE_URL + '/status/{notification_id}/{log_id}'
STATUS_LOG_SUMMARY_URL = BASE_URL + '/status/summary/{notification_id}'

USERID = 'jupyter_' + str(uuid.uuid4())[:8]

print('user_id:', USERID)

## Get signed url to upload csv

In [ ]:
csvFileName = 'NewFile_' + str(int(time.time())) + '.csv'
payload = dict(file_name = csvFileName)

response = requests.post(GET_SIGNED_URL.format(user_id = USERID), data = json.dumps(payload))
rspJson = response.json()
uploadUrl = rspJson['url']

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))
print('Copied upload URL')

## Prepare and upload csv using pre signed url

In [ ]:
from requests import Request, Session
 
rows = [['Numbers'], ['919433746949'], ['919000000009'], ['918000000008']]

buffer = io.StringIO()
writer = csv.writer(buffer)

for row in rows:
    writer.writerow(row)

content = buffer.getvalue()
content.encode('utf-8')

session = Session()
request = Request('PUT', uploadUrl, data = format(content)).prepare()
response = session.send(request)

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', response.text)

## Create Notification with bulk recipient list file

In [ ]:
from datetime import datetime  
from datetime import timedelta
date = datetime.now() + timedelta(days = 5)
message= "Your appointment is coming up on " + date.strftime("%m/%d/%Y") + " at " + date.strftime("%H:%M")

payload = dict(message = message, recipient_list_file = csvFileName, user_id = USERID)

response = requests.post(CREATE_NOTIFICATION_URL, data = json.dumps(payload))
rspJson = response.json()
notificationId = rspJson['notification_id']

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

## ** List Status Logs

### Invalid notification_id

In [ ]:
response = requests.get(LIST_STATUS_LOGS_URL.format(notification_id = 'random-notification-id'))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Valid notification_id

In [ ]:
response = requests.get(LIST_STATUS_LOGS_URL.format(notification_id = notificationId))
rspJson = response.json()
logId1 = rspJson[0]['log_id']
logId2 = rspJson[1]['log_id']

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

## ** Details Status Log

### Invalid log_id

In [ ]:
response = requests.get(DETAILS_STATUS_LOG_URL.format(notification_id = notificationId, log_id = 'random-log-id'))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Valid notification_id and log_id 1

In [ ]:
response = requests.get(DETAILS_STATUS_LOG_URL.format(notification_id = notificationId, log_id = logId1))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Valid notification_id and log_id 2

In [ ]:
response = requests.get(DETAILS_STATUS_LOG_URL.format(notification_id = notificationId, log_id = logId2))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

## ** Status Log Summary

### Invalid notification_id

In [ ]:
response = requests.get(STATUS_LOG_SUMMARY_URL.format(notification_id = 'random-notification-id'))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))

### Valid notification_id

In [ ]:
response = requests.get(STATUS_LOG_SUMMARY_URL.format(notification_id = notificationId))
rspJson = response.json()

print(response.request.method, response.request.url)
print('Status:', response.status_code)
print('Response:', json.dumps(rspJson, indent = 2))